In [1]:

import numpy as np
import pandas as pd
import glob
#from tqdm.auto import tqdm
import os
from datetime import datetime, timedelta, date
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
#import yfinance as yf
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
#from dash import Dash, html, dcc
import tkinter
import json
from tqdm import tqdm
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF8')

import warnings
warnings.filterwarnings("ignore")

from backtesting.report import Get_strategy_report, Check_strategy_list, Compute_corr, get_performance


In [2]:
import configparser

config = configparser.ConfigParser()
config.read('strategy_config.ini')

strategy_path = config['base']['path']
file_type = config['base']['file_type']
strategy_report_paths = glob.glob(strategy_path + '*' + file_type)
intraday_tradingday_count_per_month_threshold = int(config['intraday_threshold']['tradingday_count_per_month'])

tcri_path = 'C:\\Users\\012480\\Documents\\Data\\TCRI\\tcri_risk.json'


In [3]:
def get_stat_trading_df(strategy_report_path):

    stat_df = pd.read_excel(strategy_report_path, sheet_name='整體統計')
    stat_df = stat_df.iloc[:35, :3]
    stat_df.columns = ['statistics', 'value', 'ratio']

    if stat_df['statistics'].iloc[4] != '交易腳本':
        raise ValueError(strategy_name +  ' 非交易腳本 ! ')
        
    trading_df = pd.read_excel(strategy_report_path, sheet_name='交易分析')
    trading_df['進場時間'] = pd.to_datetime(trading_df['進場時間'])
    trading_df['出場時間'] = pd.to_datetime(trading_df['出場時間'].replace('--', ''))
    trading_df['獲利金額'] = trading_df['獲利金額'].replace('[\$,]', '', regex=True).astype(float)
    trading_df['買進金額'] = trading_df['進場價格'] * trading_df['交易數量'] * 1000
    trading_df = trading_df.sort_values(by='進場時間')
    
    return stat_df, trading_df
    

In [4]:

def get_tcri_df(tcri_path):
    
    with open(tcri_path, "r") as f:
        tcri_json = json.load(f)
    
    tcri_list = []

    for date_index in tcri_json.keys():
        tcri_list.append([ datetime.strptime(date_index, "%Y-%m-%d %H:%M:%S"), [stock_id[:4] for stock_id in tcri_json[date_index]]])
        
    tcri_df = pd.DataFrame(tcri_list)
    tcri_df.columns = ['date', 'stocks']

    return tcri_df
    

In [5]:

tcri_df = get_tcri_df(tcri_path)


In [12]:
def get_tcri_maintain_ratio(strategy_report_paths):

    tcri_stat_list = []

    for strategy_report_path in strategy_report_paths:
        
        stat_df, trading_df = get_stat_trading_df(strategy_report_path)
        
        ori_trading_num = len(trading_df)
        tcri_trading_num = 0

        for index in range(len(trading_df)):
            cur_stock_id = trading_df['商品名稱'].iloc[index][-8:-4]
            cur_date = trading_df['進場時間'].iloc[index]
            
            tcri_stocks = tcri_df[tcri_df['date'] <= cur_date].iloc[0]['stocks']

            if cur_stock_id not in tcri_stocks:
                tcri_trading_num += 1
            
        tcri_stat_list.append([strategy_report_path.split('\\')[1][:-5], ori_trading_num, tcri_trading_num, tcri_trading_num/ ori_trading_num ])
        
    tcri_stat_df = pd.DataFrame(tcri_stat_list)
    tcri_stat_df.columns = ['策略', '交易次數', '交易次數(剔除TCRI)', '保留比率']
    
    return tcri_stat_df


,策略,交易次數,交易次數(剔除TCRI),保留比率
0,地緣券商優化,799,622,0.778473
1,大長線破底翻barslast上市_轉機股合一,194,187,0.963918
2,大長線破底翻barslast上櫃_成長股合一轉機股,43,36,0.837209
3,法說會策略0111,512,467,0.912109
4,波段多10日聯集_0322,1401,882,0.629550
5,波段多20日聯集,1391,1041,0.748382
6,注意股10日多,143,126,0.881119
7,營收首次創新高_5趴_3000萬_24_36_12,738,429,0.581301
8,目標價_v16,1535,1535,1.000000
9,處置股10日多,171,113,0.660819


In [9]:

def get_tcri_trading_df(trading_df):

    tcri_index = []

    for index in range(len(trading_df)):
        cur_stock_id = trading_df['商品名稱'].iloc[index][-8:-4]
        cur_date = trading_df['進場時間'].iloc[index]
        
        tcri_stocks = tcri_df[tcri_df['date'] <= cur_date].iloc[0]['stocks']

        if cur_stock_id not in tcri_stocks:
            tcri_index.append(index)
            
    return trading_df[trading_df.index.isin(tcri_index)]


In [10]:

leaderboard_list = []

for strategy_report_path in tqdm(strategy_report_paths):
    
    #try:
        
        stat_df, trading_df = get_stat_trading_df(strategy_report_path)

        strategy_report_list = get_performance(strategy_report_path, stat_df, trading_df)
        strategy_tcri_report_list = get_performance(strategy_report_path, stat_df, get_tcri_trading_df(trading_df))
        
        leaderboard_list.append(strategy_report_list)
        leaderboard_list.append(strategy_tcri_report_list)
        leaderboard_list.append([np.nan]*len(strategy_report_list))

    #except:
    #    print(strategy_report_path, ' Fail ! ')

leaderboard_df = pd.DataFrame(leaderboard_list)

leaderboard_df.columns = ['策略名稱', '回測資料範圍', '回測年數', '平均持倉天數', '獲利因子', '總交易次數',
'總交易次數(一年平均)', '勝率%', '最大投入報酬率%', '最大持倉金額(萬)', '最大區間虧損率%', '風險報酬比',
'最大連續虧損月份', '賠錢年份', f'每月至少{intraday_tradingday_count_per_month_threshold}交易日有訊號', '平均每月交易日', 'A\n(X=總收/總損)\nA=(X-1)/(X+1)', 'B\n(Y=均正報酬/均負報酬)\nB=(Y-1)/(Y+1)*勝率', 'C\n(Z=平均單筆賺賠)\nC=Z/MDD', 'A+B+C']



100%|██████████| 11/11 [01:02<00:00,  5.68s/it]


In [11]:
tcri_report_path = './report/TCRI/'
leaderboard_df.to_excel(tcri_report_path + "豐神榜.xlsx", sheet_name='豐神榜績效(XQ)', index = 0)  

In [19]:
dog_df = pd.read_excel('C:\\Users\\012480\\Documents\\Data\\TCRI\\dog.xlsx')

In [22]:
dog_df[dog_df['公司'] == '2211 長榮鋼'][-50:]

,公司,事件日,序號,TCRI(年/月),事件強度,事件大類,事件小類,事件內容
135335,2211 長榮鋼,2021-07-01,001A001,5(2021/03),0,M_經營層,MT06_高管異動,財務經理葉佳全內部調動，由徐智寬接任。
138017,2211 長榮鋼,2021-06-10,001A001,5(2020/09),1,I_產業前景,IS01_營收/客戶變動或行銷策略,"2021年05月累計營收4,686,199千元，年增25%。2021年05月單月營收849,..."
142929,2211 長榮鋼,2021-05-17,001A001,5(2020/09),-1,F_市場交易,FS02_股價暴跌或異常,2021/05/17收盤價45.90元，月報酬下跌34%。
150904,2211 長榮鋼,2021-04-09,001A001,5(2020/09),0,I_產業前景,IP01_成本/產能變動或資本支出,長榮鋼旗下環保事業體，持股48.13%的水美工程，獲得經濟部工業局輔導認證，為台灣北區唯一的...
150905,2211 長榮鋼,2021-04-09,001A002,5(2020/09),0,I_產業前景,IP01_成本/產能變動或資本支出,棄物綜合處理中心，可處理工廠及醫療院所作業衍生的有害事業廢棄物，處理能量為每年2萬噸，預計將...
150906,2211 長榮鋼,2021-04-09,001A003,5(2020/09),0,I_產業前景,IP01_成本/產能變動或資本支出,線，2022年完成，2023年商業運轉，產能將擴充50%至一年3萬噸。水美旗下客戶有日月光投...
150907,2211 長榮鋼,2021-04-09,001A004,5(2020/09),0,I_產業前景,IP01_成本/產能變動或資本支出,科、台塑、台塑石化、亞東石化、長興化工、必治妥、杏輝製藥、中研院、國防部、環保局、台大醫院、...
150908,2211 長榮鋼,2021-04-09,001A005,5(2020/09),0,I_產業前景,IP01_成本/產能變動或資本支出,院等。
150909,2211 長榮鋼,2021-04-09,002A001,5(2020/09),1,I_產業前景,IS01_營收/客戶變動或行銷策略,"2021年03月累計營收3,048,197千元，年增25%。2021年03月單月營收1,20..."
156982,2211 長榮鋼,2021-03-12,001A001,5(2020/09),-1,M_經營層,MC02_環保議題,長榮鋼鐵公告新營廠從事金屬結構及建築組件製造業，領有固定污染源操作許可證，噴砂機(E002)...
